In [91]:
import numpy as np
from cython_weave import *
import tensorflow as tf
#from fancy_conv import array_weave_forward

In [149]:
def create_I_matrix(N, weave_param):
    N_arr = np.arange(1,N**2 + 1).reshape([N,N]).astype(float)
    N_weave,_ = array_weave_fast_forward(N_arr, weave_param)
    N_weave[np.where(N_weave == 0)] = -1
    num_rows, num_cols = N_weave.shape
    rows = np.zeros([num_rows,num_cols])
    cols = np.zeros([num_rows,num_cols])
    for i in range(num_cols):
        for j in range(num_rows):
            i_j_loc = np.where(N_weave[i,j] == N_arr)
            if len(i_j_loc[0]) > 0:
                rows[i,j] = int(i_j_loc[0])
                cols[i,j] = int(i_j_loc[1])
            else:
                rows[i,j] = 0

                cols[i,j] = N
    return (rows.astype(int),cols.astype(int))

def create_full_I_matrix(N, num_images, num_filters, weave_param):
    """
    Creates an matrix of size (num_images, num_filters, height, width, input_tensor_dimension (4))
    
    """
    tensor_dimension = 4
    N_arr = np.arange(1,N**2 * num_filters * num_images + 1).reshape([num_images,num_filters,N,N]).astype(float)
    N_weave,_ = array_weave_fast_forward(N_arr, weave_param)
    N_weave[np.where(N_weave == 0)] = -1
    _,_,num_rows, num_cols = N_weave.shape
    print(N_weave.shape)
    
    I_full = np.zeros([num_images,num_filters,num_rows,num_cols,tensor_dimension])
    
    for img in range(num_images):
        print(img)
        for fil in range(num_filters):
            for i in range(num_cols):
                for j in range(num_rows):
                    i_j_loc = np.where(N_weave[img,fil,i,j] == N_arr)
                    if len(i_j_loc[0]) > 0:
                        I_full[img,fil,i,j] = i_j_loc
                    else:
                        I_full[img,fil,i,j] = np.array([0,0,0,N])
    return I_full.astype(int)

In [152]:
A = np.array([[[1,2,3],[4,5,6]],[[7,8,9],[10,11,12]]])
print(A)
test = np.where(A == A)

[[[ 1  2  3]
  [ 4  5  6]]

 [[ 7  8  9]
  [10 11 12]]]


array([0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1])

In [140]:
weave_param = {'num_zeros':2, 'filter_size':3}
N = 5
num_images = 4
num_filters = 3
print(create_full_I_matrix(N, num_images, num_filters, weave_param))


(4, 3, 13, 13)
[[[[[0 0 0 5]
    [0 0 0 3]
    [0 0 0 5]
    ...
    [0 0 0 5]
    [0 0 0 1]
    [0 0 0 5]]

   [[0 0 3 0]
    [0 0 3 3]
    [0 0 0 5]
    ...
    [0 0 0 5]
    [0 0 3 1]
    [0 0 3 4]]

   [[0 0 0 5]
    [0 0 0 5]
    [0 0 0 5]
    ...
    [0 0 0 5]
    [0 0 0 5]
    [0 0 0 5]]

   ...

   [[0 0 0 5]
    [0 0 0 5]
    [0 0 0 5]
    ...
    [0 0 0 5]
    [0 0 0 5]
    [0 0 0 5]]

   [[0 0 1 0]
    [0 0 1 3]
    [0 0 0 5]
    ...
    [0 0 0 5]
    [0 0 1 1]
    [0 0 1 4]]

   [[0 0 0 5]
    [0 0 4 3]
    [0 0 0 5]
    ...
    [0 0 0 5]
    [0 0 4 1]
    [0 0 0 5]]]


  [[[0 0 0 5]
    [0 1 0 3]
    [0 0 0 5]
    ...
    [0 0 0 5]
    [0 1 0 1]
    [0 0 0 5]]

   [[0 1 3 0]
    [0 1 3 3]
    [0 0 0 5]
    ...
    [0 0 0 5]
    [0 1 3 1]
    [0 1 3 4]]

   [[0 0 0 5]
    [0 0 0 5]
    [0 0 0 5]
    ...
    [0 0 0 5]
    [0 0 0 5]
    [0 0 0 5]]

   ...

   [[0 0 0 5]
    [0 0 0 5]
    [0 0 0 5]
    ...
    [0 0 0 5]
    [0 0 0 5]
    [0 0 0 5]]

   [[0 1 1 0]
    [0 1 1 3]

In [141]:
weave_param = {'num_zeros':2, 'filter_size':3}
I = create_I_matrix(5,weave_param)
print(I)

(array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [3, 3, 0, 3, 3, 0, 3, 0, 3, 3, 0, 3, 3],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0],
       [4, 4, 0, 4, 4, 0, 4, 0, 4, 4, 0, 4, 4],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 2, 0, 0, 2, 0, 0, 0, 2, 0, 0, 2, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 3, 0, 0, 3, 0, 0, 0, 3, 0, 0, 3, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1],
       [0, 4, 0, 0, 4, 0, 0, 0, 4, 0, 0, 4, 0]]), array([[5, 3, 5, 5, 4, 5, 5, 5, 0, 5, 5, 1, 5],
       [0, 3, 5, 1, 4, 5, 2, 5, 0, 3, 5, 1, 4],
       [5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5],
       [5, 3, 5, 5, 4, 5, 5, 5, 0, 5, 5, 1, 5],
       [0, 3, 5, 1, 4, 5, 2, 5, 0, 3, 5, 1, 4],
       [5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5],
       [5, 3, 5, 5, 4, 5, 5, 5, 0, 5, 5, 1, 5],
       [5, 5, 5, 5, 5, 5, 5, 5, 5, 5,

In [150]:
# Input
N = 10
n_fil = 50
n_img = 10

a = tf.constant(np.arange(1,n_fil*n_img*N**2+1).reshape(n_img,n_fil,N,N), tf.float32)
a = tf.concat((a, np.zeros([n_img,n_fil,N,1])), axis = 3)
image_indx = create_full_I_matrix(N, n_img, n_fil, weave_param)

# collect_indx = np.stack([rows,cols])
# print('collect_indx', collect_indx.shape)
# filter_indx = np.stack([collect_indx]* n_fil)
# print('filter_indx', filter_indx.shape)
# image_indx = np.stack([filter_indx]*n_img)
# print('image_indx', image_indx.shape)
# image_indx = np.transpose(image_indx, [0,1,3,4,2])
# print('image_indx', image_indx.shape)
x = tf.placeholder('float32', (n_img,n_fil,16, 16))

with tf.Session() as sess:
    print(tf.shape(a))
    r = sess.run(tf.gather_nd(a, image_indx))
print(r)

(10, 50, 28, 28)
0
1
2
3
4
5
6
7
8
9


KeyboardInterrupt: 

In [102]:
# Input
N = 6
n_fil = 5

a = tf.constant(np.arange(1,n_fil*N**2+1).reshape(n_fil,N,N), tf.float32)
a = tf.concat((a, np.zeros([n_fil,N,1])), axis = 2)
rows,cols = create_I_matrix(N,weave_param)

for img in range(n_img):
    for layer in range(n_fil):
    

collect_indx = np.stack([rows,cols])
print('collect_indx', collect_indx.shape)
#filter_indx = np.stack([collect_indx]* n_fil)
#print('filter_indx', filter_indx.shape)
x = tf.placeholder('float32', (n_img,n_fil,16, 16))

with tf.Session() as sess:
    print(tf.shape(a))
    r = sess.run(tf.gather_nd(a, image_indx))
print(r)

('collect_indx', (2, 16, 16))
Tensor("Shape_8:0", shape=(3,), dtype=int32)


InvalidArgumentError: flat indices[12799, :] = [0, 6] does not index into param (shape: [5,6,7]).
	 [[Node: GatherNd_64 = GatherNd[Tindices=DT_INT64, Tparams=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"](concat_48, GatherNd_64/indices)]]

Caused by op u'GatherNd_64', defined at:
  File "/Users/Tim/anaconda2/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/Users/Tim/anaconda2/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/Users/Tim/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/Tim/anaconda2/lib/python2.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/Tim/anaconda2/lib/python2.7/site-packages/ipykernel/kernelapp.py", line 478, in start
    self.io_loop.start()
  File "/Users/Tim/anaconda2/lib/python2.7/site-packages/zmq/eventloop/io